# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-27-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-26-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-27 05:23:41,33.93911,67.709953,54750,2389,47459,4902.0,Afghanistan,140.643034,4.363470
1,NaN,NaN,NaN,Albania,2021-01-27 05:23:41,41.15330,20.168300,73691,1332,44880,27479.0,Albania,2560.671346,1.807548
2,NaN,NaN,NaN,Algeria,2021-01-27 05:23:41,28.03390,1.659600,106097,2871,72336,30890.0,Algeria,241.948635,2.706014
3,NaN,NaN,NaN,Andorra,2021-01-27 05:23:41,42.50630,1.521800,9638,97,8774,767.0,Andorra,12473.953278,1.006433
4,NaN,NaN,NaN,Angola,2021-01-27 05:23:41,-11.20270,17.873900,19553,462,17388,1703.0,Angola,59.492608,2.362809


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,53984,54062,54141,54278,54403,54483,54559,54595,54672,54750
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,67690,67982,68568,69238,69916,70655,71441,72274,72812,73691
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,103833,104092,104341,104606,104852,105124,105369,105596,105854,106097


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2339,2343,2346,2354,2363,2370,2373,2378,2385,2389
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1277,1281,1287,1291,1296,1303,1310,1315,1324,1332
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2836,2840,2843,2849,2853,2856,2861,2863,2866,2871


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,45868,46359,46554,46759,46887,46912,46943,47298,47365,47459
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,40453,40870,41464,41969,42426,42895,43384,43890,44291,44880
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,70554,70747,70933,71127,71343,71552,71755,71945,72143,72336


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5184,5198,5227,5257,5270,5327,5358,5376,5407,5440
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,16251,16346,16513,16653,16798,16981,17128,17256,17333,17496
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1729,1730,1738,1760,1778,1793,1805,1827,1834,1882


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,55,55,55,55,61,61,62,62,62,65
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,182,182,183,184,188,188,189,189,189,216
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,36,36,36,36,36,37,38,38,38,40


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
651,1001.0,Autauga,Alabama,US,2021-01-27 05:23:41,32.539527,-86.644082,5440,65,0,5375.0,"Autauga, Alabama, US",9737.063488,1.194853
688,1075.0,Lamar,Alabama,US,2021-01-27 05:23:41,33.779950,-88.096680,1213,24,0,1189.0,"Lamar, Alabama, US",8786.671496,1.978566
689,1077.0,Lauderdale,Alabama,US,2021-01-27 05:23:41,34.901719,-87.656247,7986,112,0,7874.0,"Lauderdale, Alabama, US",8612.192518,1.402454


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,25445583,425252,0
India,10689527,153724,10359305
Brazil,8933356,218878,7896290


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,25445583,425252,0,25020331,2021-01-27 05:23:41,37.936303,-91.379001
India,10689527,153724,10359305,176498,2021-01-27 05:23:41,23.088275,81.806127
Brazil,8933356,218878,7896290,818188,2021-01-27 05:23:41,-12.669522,-48.480493
Russia,3716228,69391,3140930,505907,2021-01-27 05:23:41,54.546312,62.120860
United Kingdom,3700268,100359,9160,3590749,2021-01-27 05:23:41,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3235045,38268,0
Texas,2290621,35420,0
Florida,1667763,25673,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3235045,38268,0,3196777,2021-01-27 05:23:41,37.843962,-120.728594
Texas,2290621,35420,0,2255201,2021-01-27 05:23:41,31.660643,-98.653069
Florida,1667763,25673,0,1642090,2021-01-27 05:23:41,28.940755,-82.700744
New York,1358707,42726,0,1315981,2021-01-27 05:23:41,42.544151,-75.474183
Illinois,1108430,20853,0,1087577,2021-01-27 05:23:41,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1085488,15614,0
Arizona,Maricopa,455830,7053,0
Illinois,Cook,447295,9280,0
Florida,Miami-Dade,362601,4759,0
Texas,Harris,304333,4060,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1085488,15614,0,1069874,2021-01-27 05:23:41,34.308284,-118.228241,6037.0
Arizona,Maricopa,455830,7053,0,448777,2021-01-27 05:23:41,33.348359,-112.491815,4013.0
Illinois,Cook,447295,9280,0,438015,2021-01-27 05:23:41,41.841448,-87.816588,17031.0
Florida,Miami-Dade,362601,4759,0,357842,2021-01-27 05:23:41,25.611236,-80.551706,12086.0
Texas,Harris,304333,4060,0,300273,2021-01-27 05:23:41,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-24,54595,72274,105596,9549,19399,198,1867223,166036,28777,404714,...,3657857,37633,78375,1,123709,1548,155414,2118,45337,31320
2021-01-25,54672,72812,105854,9596,19476,201,1874801,166094,28780,405723,...,3680101,38041,78429,1,124112,1549,155884,2118,46146,31646
2021-01-26,54750,73691,106097,9638,19553,201,1885210,166232,28786,407140,...,3700268,38680,78471,1,124525,1551,156393,2119,47622,32004


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-24,2378,1315,2863,96,459,6,46827,3039,909,7418,...,98129,376,621,0,1148,35,1791,615,639,1005
2021-01-25,2385,1324,2866,97,461,6,47034,3047,909,7451,...,98723,390,621,0,1154,35,1796,615,660,1075
2021-01-26,2389,1332,2871,97,462,6,47253,3052,909,7515,...,100359,401,621,0,1159,35,1803,615,672,1103


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-24,47298,43890,71945,8652,17266,163,1656149,154708,25999,381654,...,8946,29270,76815,1,115426,1411,144910,1423,35960,22250
2021-01-25,47365,44291,72143,8724,17327,170,1666527,154932,26003,383200,...,9036,30143,76862,1,116365,1425,145639,1424,38639,22877
2021-01-26,47459,44880,72336,8774,17388,170,1674555,155404,26018,384926,...,9160,30861,76912,1,116785,1430,146279,1424,40362,23413


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5184,5198,5227,5257,5270,5327,5358,5376,5407,5440
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1180,1182,1184,1190,1196,1197,1199,1202,1206,1213
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,7546,7600,7631,7686,7733,7798,7847,7871,7946,7986
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2436,2442,2452,2466,2491,2507,2533,2541,2558,2569
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,12465,12515,12603,12749,12901,13036,13137,13205,13261,13378


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-24,441170,53315,0,722574,284066,3187475,386285,237815,74575,49,...,708717,2250421,336405,11033,2341,472447,300198,115307,581377,50898
2021-01-25,443009,53399,0,727895,284702,3213222,387500,243632,75191,49,...,710427,2269424,337264,11165,2350,478619,302141,115839,582477,51062
2021-01-26,445909,53487,0,732643,287187,3235045,388620,244899,75490,49,...,712406,2290621,338675,11285,2373,483326,303482,116978,584041,51152


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-01-24        5376   17256    1827  2223   5473    1035   1744   11521   
2021-01-25        5407   17333    1834  2229   5485    1046   1748   11555   
2021-01-26        5440   17496    1882  2247   5517    1058   1759   11626   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-24         3042     1657  ...    2521    541     2828      641   
2021-01-25         3054     1660  ...    2518    543     2838      641   
2021-01-26         3085     1674  ...    2517    545     2841      644   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-24           3411  2931  1900          0      858    610  
2021-01-25           3418  2981  1913          0      861    610  
2021-01-26           3421  2993  1925          0      863    609  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-24,6660,259,0,12238,4606,37121,5505,6819,1041,0,...,8859,34911,1595,170,24,6078,4114,1895,6190,571
2021-01-25,6662,259,0,12239,4650,37543,5512,6911,1049,0,...,8970,35090,1597,171,24,6081,4148,1899,6199,571
2021-01-26,6896,260,0,12448,4690,38268,5517,6934,1065,0,...,9162,35420,1613,171,24,6174,4167,1928,6264,596


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-01-24          62     189      38   48     86      28     60     206   
2021-01-25          62     189      38   48     86      28     60     206   
2021-01-26          65     216      40   51     94      29     60     228   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-24           70       31  ...      18     10       18        7   
2021-01-25           70       31  ...      18     10       18        7   
2021-01-26           75       33  ...      20     11       19        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-24             30     5    12          0       25      4  
2021-01-25             30     5    12          0       25      4  
2021-01-26             32     6    12          0       25      4  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-24,0.000660,0.011660,0.002154,0.005264,0.001652,0.015385,0.002702,0.000765,0.000382,0.002979,...,0.008300,0.018016,0.000741,0.0,0.003765,0.000000,0.002632,0.000000,0.016707,0.010094
2021-01-25,0.001410,0.007444,0.002443,0.004922,0.003969,0.015152,0.004058,0.000349,0.000104,0.002493,...,0.006081,0.010842,0.000689,0.0,0.003258,0.000646,0.003024,0.000000,0.017844,0.010409
2021-01-26,0.001427,0.012072,0.002296,0.004377,0.003954,0.000000,0.005552,0.000831,0.000208,0.003493,...,0.005480,0.016798,0.000536,0.0,0.003328,0.001291,0.003265,0.000472,0.031985,0.011313


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-24,0.002107,0.003817,0.000699,0.000000,0.004376,0.0,0.001926,0.001648,0.0,0.003925,...,0.006266,0.024523,0.0,NaN,0.005254,0.0,0.004487,0.0,0.019139,0.031828
2021-01-25,0.002944,0.006844,0.001048,0.010417,0.004357,0.0,0.004421,0.002632,0.0,0.004449,...,0.006053,0.037234,0.0,NaN,0.005226,0.0,0.002792,0.0,0.032864,0.069652
2021-01-26,0.001677,0.006042,0.001745,0.000000,0.002169,0.0,0.004656,0.001641,0.0,0.008589,...,0.016572,0.028205,0.0,NaN,0.004333,0.0,0.003898,0.0,0.018182,0.026047


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-24,0.007562,0.011663,0.002648,0.007452,0.000986,0.006173,0.005758,0.001119,0.000539,0.003698,...,0.013596,0.025327,0.000482,0.0,0.000000,0.000000,0.004248,0.000000,0.048611,0.040838
2021-01-25,0.001417,0.009136,0.002752,0.008322,0.003533,0.042945,0.006266,0.001448,0.000154,0.004051,...,0.010060,0.029826,0.000612,0.0,0.008135,0.009922,0.005031,0.000703,0.074499,0.028180
2021-01-26,0.001985,0.013298,0.002675,0.005731,0.003521,0.000000,0.004817,0.003046,0.000577,0.004504,...,0.013723,0.023820,0.000651,0.0,0.003609,0.003509,0.004394,0.000000,0.044592,0.023430


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-24,0.003932,0.003104,NaN,0.010089,0.003785,0.006139,0.003426,0.00000,0.008929,0.0,...,0.004025,0.003544,0.004527,0.011459,0.000000,0.008091,0.000000,0.004837,0.002369,0.006227
2021-01-25,0.004168,0.001576,NaN,0.007364,0.002239,0.008078,0.003145,0.02446,0.008260,0.0,...,0.002413,0.008444,0.002553,0.011964,0.003845,0.013064,0.006472,0.004614,0.001892,0.003222
2021-01-26,0.006546,0.001648,NaN,0.006523,0.008728,0.006792,0.002890,0.00520,0.003977,0.0,...,0.002786,0.009340,0.004184,0.010748,0.009787,0.009835,0.004438,0.009833,0.002685,0.001763


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-24      0.003359  0.007473  0.012188  0.000000  0.002014  0.001936   
2021-01-25      0.005766  0.004462  0.003831  0.002699  0.002193  0.010628   
2021-01-26      0.006103  0.009404  0.026172  0.008075  0.005834  0.011472   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-24      0.007510  0.002175  0.002637  0.003634  ...  0.005584   
2021-01-25      0.002294  0.002951  0.003945  0.001811  ... -0.001190   
2021-01-26      0.006293  0.006145  0.010151  0.008434  ... -0.000397   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-24      0.001852  0.001416  0.001562   0.007979  0.021254  0.006889   
2021-01-25      0.003697  0.003536  0.000000   0.002052  0.017059  0.006842   
2021-01-26      0.003683  0.001057  0.004680   0.000878  0.004025  0.006273   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-24            NaN  0.005862  0.001642  
2021-01-25            NaN  0.003497  0.000000  
2021-01-26            NaN  0.002323 -0.001639  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-24,0.000451,0.000000,NaN,0.005588,0.009424,0.007463,0.004196,0.000000,0.004826,NaN,...,0.004536,0.005356,0.008217,0.000000,0.0,-0.000165,0.000000,0.012286,0.000970,0.000000
2021-01-25,0.000300,0.000000,NaN,0.000082,0.009553,0.011368,0.001272,0.013492,0.007685,NaN,...,0.012530,0.005127,0.001254,0.005882,0.0,0.000494,0.008264,0.002111,0.001454,0.000000
2021-01-26,0.035125,0.003861,NaN,0.017077,0.008602,0.019311,0.000907,0.003328,0.015253,NaN,...,0.021405,0.009404,0.010019,0.000000,0.0,0.015294,0.004581,0.015271,0.010486,0.043783


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour    Bibb    Blount   Bullock   
2021-01-24      0.000000  0.000000  0.000000  0.0000  0.000000  0.000000   
2021-01-25      0.000000  0.000000  0.000000  0.0000  0.000000  0.000000   
2021-01-26      0.048387  0.142857  0.052632  0.0625  0.093023  0.035714   

Province_State                                       ...   Wyoming         \
Admin2         Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2021-01-24        0.0  0.000000  0.000000  0.000000  ...  0.000000    0.0   
2021-01-25        0.0  0.000000  0.000000  0.000000  ...  0.000000    0.0   
2021-01-26        0.0  0.106796  0.071429  0.064516  ...  0.111111    0.1   

Province_State                                                                \
Admin2          Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2021-01-24      0.000000      0.0   0.000000   0.0   0.0        NaN      0.0   
2021-01-25      0.000000      0.0   0.000000   0.0   0.0        NaN      0.0   
2021-01-26      0.055556      0.0   0.066667   0.2   0.0        NaN      0.0   

Province_State         
Admin2         Weston  
2021-01-24        0.0  
2021-01-25        0.0  
2021-01-26        0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-24,0.001130,0.011082,0.002285,0.006440,0.003241,0.010483,0.003985,0.001037,0.000300,0.003666,...,0.009274,0.021387,0.000694,0.0,0.003947,0.000352,0.002846,0.000198,0.023016,0.013962
2021-01-25,0.001270,0.009263,0.002364,0.005681,0.003605,0.012818,0.004022,0.000693,0.000202,0.003080,...,0.007677,0.016115,0.000692,0.0,0.003602,0.000499,0.002935,0.000099,0.020430,0.012185
2021-01-26,0.001348,0.010668,0.002330,0.005029,0.003779,0.006409,0.004787,0.000762,0.000205,0.003286,...,0.006579,0.016456,0.000614,0.0,0.003465,0.000895,0.003100,0.000286,0.026208,0.011749


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-24,0.002142,0.004400,0.001112,0.008992,0.007004,0.000098,0.002886,0.001808,2.051371e-12,0.005565,...,0.010405,0.024077,0.000431,NaN,0.005417,1.783361e-45,0.005719,0.000612,0.021783,0.031956
2021-01-25,0.002543,0.005622,0.001080,0.009704,0.005681,0.000049,0.003653,0.002220,1.025686e-12,0.005007,...,0.008229,0.030655,0.000215,NaN,0.005322,8.916805e-46,0.004256,0.000306,0.027323,0.050804
2021-01-26,0.002110,0.005832,0.001412,0.004852,0.003925,0.000024,0.004155,0.001931,5.128428e-13,0.006798,...,0.012400,0.029430,0.000108,NaN,0.004827,4.458403e-46,0.004077,0.000153,0.022753,0.038425


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-24,0.004518,0.011510,0.002758,0.007298,0.002707,0.005190,0.006085,0.001404,0.000494,0.004317,...,0.014197,0.027630,0.000549,0.0,0.002131,0.000445,0.004808,0.000045,0.047975,0.037972
2021-01-25,0.002967,0.010323,0.002755,0.007810,0.003120,0.024067,0.006176,0.001426,0.000324,0.004184,...,0.012129,0.028728,0.000580,0.0,0.005133,0.005184,0.004919,0.000374,0.061237,0.033076
2021-01-26,0.002476,0.011811,0.002715,0.006771,0.003320,0.012034,0.005496,0.002236,0.000450,0.004344,...,0.012926,0.026274,0.000615,0.0,0.004371,0.004346,0.004657,0.000187,0.052915,0.028253


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-24,0.005730,0.003890,NaN,0.010447,0.005446,0.006751,0.004154,0.002167,0.009119,1.020008e-95,...,0.004816,0.005814,0.005338,0.012819,0.003653,0.009112,0.003385,0.007085,0.003037,0.004172
2021-01-25,0.004949,0.002733,NaN,0.008906,0.003843,0.007414,0.003650,0.013314,0.008690,5.100039e-96,...,0.003615,0.007129,0.003946,0.012392,0.003749,0.011088,0.004929,0.005850,0.002464,0.003697
2021-01-26,0.005748,0.002190,NaN,0.007714,0.006286,0.007103,0.003270,0.009257,0.006333,2.550019e-96,...,0.003200,0.008235,0.004065,0.011570,0.006768,0.010461,0.004683,0.007841,0.002575,0.002730


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-24      0.004988  0.008335  0.009975  0.002721  0.003524  0.005410   
2021-01-25      0.005377  0.006399  0.006903  0.002710  0.002858  0.008019   
2021-01-26      0.005740  0.007901  0.016538  0.005393  0.004346  0.009746   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-24      0.005894  0.003912  0.006551  0.004921  ...  0.003713   
2021-01-25      0.004094  0.003432  0.005248  0.003366  ...  0.001262   
2021-01-26      0.005193  0.004788  0.007699  0.005900  ...  0.000432   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-24      0.001138  0.001527  0.001378   0.005273  0.013606  0.005394   
2021-01-25      0.002418  0.002531  0.000689   0.003662  0.015333  0.006118   
2021-01-26      0.003050  0.001794  0.002685   0.002270  0.009679  0.006195   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-24           -1.0  0.003883  0.001264  
2021-01-25           -1.0  0.003690  0.000632  
2021-01-26           -1.0  0.003006 -0.000504  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-24,0.010756,0.008372,NaN,0.010874,0.008383,0.011349,0.003969,0.001613,0.004932,NaN,...,0.006213,0.008317,0.009409,0.003763,2.073205e-08,0.005836,0.003656,0.009984,0.004511,0.005619
2021-01-25,0.005528,0.004186,NaN,0.005478,0.008968,0.011359,0.002620,0.007552,0.006309,NaN,...,0.009371,0.006722,0.005331,0.004823,1.036603e-08,0.003165,0.005960,0.006047,0.002983,0.002810
2021-01-26,0.020326,0.004023,NaN,0.011277,0.008785,0.015335,0.001764,0.005440,0.010781,NaN,...,0.015388,0.008063,0.007675,0.002411,5.183013e-09,0.009229,0.005270,0.010659,0.006734,0.023296


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                   \
Admin2           Autauga   Baldwin   Barbour     Bibb    Blount   Bullock   
2021-01-24      0.010919  0.002996  0.010243  0.00132  0.002303  0.000002   
2021-01-25      0.005460  0.001498  0.005122  0.00066  0.001152  0.000001   
2021-01-26      0.026923  0.072178  0.028877  0.03158  0.047087  0.017858   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-24      0.006405  0.002042  0.000477  0.008376  ...  0.008339   
2021-01-25      0.003203  0.001021  0.000238  0.004188  ...  0.004170   
2021-01-26      0.001601  0.053908  0.035833  0.034352  ...  0.057640   

Province_State                                                         \
Admin2            Platte  Sheridan      Sublette Sweetwater     Teton   
2021-01-24      0.001736  0.000919  1.649062e-07   0.010142  0.003906   
2021-01-25      0.000868  0.000460  8.245310e-08   0.005071  0.001953   
2021-01-26      0.050434  0.028008  4.122655e-08   0.035869  0.100977   

Province_State                                               
Admin2             Uinta Unassigned  Washakie        Weston  
2021-01-24      0.026752  -0.501961  0.005899  1.748403e-06  
2021-01-25      0.013376  -0.501961  0.002950  8.742015e-07  
2021-01-26      0.006688  -0.501961  0.001475  4.371007e-07  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210127,AK,51951,NaN,NaN,NaN,totalTestsViral,1467400,49.0,1200.0,...,51951,1,6,f860fa6612bfb8b32661335f13f2beafdac5ae55,0,0,0,0,0,NaN
1,20210127,AL,449086,94119.0,1751068.0,NaN,totalTestsPeopleViral,2106035,2122.0,41483.0,...,2200154,276,168,ea8318ddf483ea583e591140d24dbb43210ec36f,0,0,0,0,0,NaN
2,20210127,AR,288964,58097.0,2149071.0,NaN,totalTestsViral,2379938,1029.0,13447.0,...,2438035,52,46,fc436a063fa7942d79458331987059714ae58d15,0,0,0,0,0,NaN
3,20210127,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,33161a6289bcb7efb40e086dc073baec0d709aa9,0,0,0,0,0,NaN
4,20210127,AZ,738561,46062.0,2702143.0,NaN,totalTestsViral,6504407,4250.0,51037.0,...,3440704,195,380,c32027cb5617b7520fafdc4979f464f4e464be2f,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210127,AK,51951.0,NaN,NaN,NaN,totalTestsViral,1467400.0,49.0,1200.0,...,51951,1,6,f860fa6612bfb8b32661335f13f2beafdac5ae55,0,0,0,0,0,NaN
1,20210127,AL,449086.0,94119.0,1751068.0,NaN,totalTestsPeopleViral,2106035.0,2122.0,41483.0,...,2200154,276,168,ea8318ddf483ea583e591140d24dbb43210ec36f,0,0,0,0,0,NaN
2,20210127,AR,288964.0,58097.0,2149071.0,NaN,totalTestsViral,2379938.0,1029.0,13447.0,...,2438035,52,46,fc436a063fa7942d79458331987059714ae58d15,0,0,0,0,0,NaN
3,20210127,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,33161a6289bcb7efb40e086dc073baec0d709aa9,0,0,0,0,0,NaN
4,20210127,AZ,738561.0,46062.0,2702143.0,NaN,totalTestsViral,6504407.0,4250.0,51037.0,...,3440704,195,380,c32027cb5617b7520fafdc4979f464f4e464be2f,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-27,AK,51951,NaN,NaN,NaN,totalTestsViral,1467400,49.0,1200.0,NaN,...,51951,1,6,f860fa6612bfb8b32661335f13f2beafdac5ae55,0,0,0,0,0,NaN
2021-01-27,AL,449086,94119.0,1751068.0,NaN,totalTestsPeopleViral,2106035,2122.0,41483.0,NaN,...,2200154,276,168,ea8318ddf483ea583e591140d24dbb43210ec36f,0,0,0,0,0,NaN
2021-01-27,AR,288964,58097.0,2149071.0,NaN,totalTestsViral,2379938,1029.0,13447.0,321.0,...,2438035,52,46,fc436a063fa7942d79458331987059714ae58d15,0,0,0,0,0,NaN
2021-01-27,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,33161a6289bcb7efb40e086dc073baec0d709aa9,0,0,0,0,0,NaN
2021-01-27,AZ,738561,46062.0,2702143.0,NaN,totalTestsViral,6504407,4250.0,51037.0,1024.0,...,3440704,195,380,c32027cb5617b7520fafdc4979f464f4e464be2f,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-27,AK,51951.0,NaN,NaN,NaN,totalTestsViral,1467400.0,49.0,1200.0,NaN,...,51951,1,6,f860fa6612bfb8b32661335f13f2beafdac5ae55,0,0,0,0,0,NaN
2021-01-27,AL,449086.0,94119.0,1751068.0,NaN,totalTestsPeopleViral,2106035.0,2122.0,41483.0,NaN,...,2200154,276,168,ea8318ddf483ea583e591140d24dbb43210ec36f,0,0,0,0,0,NaN
2021-01-27,AR,288964.0,58097.0,2149071.0,NaN,totalTestsViral,2379938.0,1029.0,13447.0,321.0,...,2438035,52,46,fc436a063fa7942d79458331987059714ae58d15,0,0,0,0,0,NaN
2021-01-27,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,33161a6289bcb7efb40e086dc073baec0d709aa9,0,0,0,0,0,NaN
2021-01-27,AZ,738561.0,46062.0,2702143.0,NaN,totalTestsViral,6504407.0,4250.0,51037.0,1024.0,...,3440704,195,380,c32027cb5617b7520fafdc4979f464f4e464be2f,0,0,0,0,0,NaN
